<a href="https://colab.research.google.com/github/UninaLabs-EO/ClaudioParserAIS/blob/master/ICEVISION_v2_rdp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Torch - Torchvision - IceVision - IceData - MMDetection - YOLOv5 - EfficientDet Installation
!wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh

# Choose your installation target: cuda11 or cuda10 or cpu
!bash icevision_install.sh cuda11

--2022-07-01 12:52:41--  https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2766 (2.7K) [text/plain]
Saving to: ‘icevision_install.sh’

icevision_install.s 100%[===================>]   2.70K  --.-KB/s    in 0s      

2022-07-01 12:52:42 (42.0 MB/s) - ‘icevision_install.sh’ saved [2766/2766]

Installing icevision + dependencices for cuda11
- Installing torch and its dependencies
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 1.3 MB/s eta 0:16:51tcmalloc: large alloc 1147494400 bytes == 0x390c4000 @  0x7efd9130f615 0x

In [2]:
!pip install pillow==8.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 30.6 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [1]:
# Restart kernel after installation
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

In [2]:
from icevision.all import *
from fastai.callback.wandb import *
from fastai.callback.tracker import SaveModelCallback
from pathlib import Path
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
path = Path('/content/drive/MyDrive/ULTIMO_DATASET/scale3/CL012/B3/')
train_parser = parsers.COCOBBoxParser(annotations_filepath = path / "Annotations/LAST_SWD_Train.json", img_dir = path / "B3_Train")
# valid_parser = parsers.COCOBBoxParser(annotations_filepath = path / "Annotations/valid.json", img_dir = path / "Immagini_test")
valid_parser = parsers.COCOBBoxParser(annotations_filepath = path / "Annotations/LAST_SWD_Test.json", img_dir = path / "B3_Test")

train_records, *_ = train_parser.parse(data_splitter=SingleSplitSplitter(), autofix=False)
valid_records, *_ = valid_parser.parse(data_splitter=SingleSplitSplitter(), autofix=False)

In [ ]:
print(f'Mappate le classi del dataset:')
print(train_parser.class_map)
print(f'Numero di record buoni ====> TRAIN:{len(train_records)}, VALID:{len(valid_records)}')

In [ ]:
# train_records.get_by_record_id(record_id=611)

In [ ]:
# Transforms: DATA AUGMENTATION.
# size is set to 384 because EfficientDet requires its inputs to be divisible by 128
image_size = 768
toFloat = tfms.A.ToFloat(max_value=255.0, always_apply=False, p=1.0)

# train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=image_size, presize=1024), tfms.A.Normalize()])
# valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size), tfms.A.Normalize()])


train_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size),  toFloat])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size),  toFloat])

In [ ]:
# image_size = 768
FLIP = [tfms.A.Flip(always_apply=False, p=0.5),
tfms.A.HorizontalFlip(always_apply=False, p=0.5)]
CLAHE = tfms.A.CLAHE(always_apply=False, p=0.05, clip_limit=(1, 4),
tile_grid_size=(8, 8))
Equalize = tfms.A.Equalize(always_apply=False, p=0.05, mode='pil',
by_channels=True)
RandomBrightnessContrast =tfms.A.RandomBrightnessContrast(always_apply=False, p=0.25,
brightness_limit=(-0.05, 0.05), contrast_limit=(-0.05, 0.05),
brightness_by_max=True)
Rotate = tfms.A.Rotate(always_apply=False, p=0.33, limit=(-25, 25),
interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None)
Blur = tfms.A.Blur(always_apply=False, p=0.1, blur_limit=(3, 5))
toFloat = tfms.A.ToFloat(max_value=255.0, always_apply=False, p=1.0)


train_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size),Equalize,FLIP[0],FLIP[1],CLAHE,RandomBrightnessContrast,Rotate,toFloat])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size),tfms.A.ToFloat(max_value=255.0,
always_apply=False, p=1.0)])

In [ ]:
# Datasets
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

In [ ]:
import numpy as np

def My_denormalize(img, mean=None, std=None, max_pixel_value=255):
  return np.around(img * max_pixel_value).astype(np.uint8)

# Show an element of the train_ds with augmentation transformations applied
idx = np.random.randint(low=0, high=500)
print('idx:',idx)
samples = [train_ds[idx] for _ in range(10)]
show_samples(samples, ncols=5, denormalize_fn=My_denormalize)
# No data augmentation applied, at least for now.
record = train_ds[idx].as_dict()
img = record['common']['img']
print('indicator:',img.mean())
# plt.imshow(img)

In [ ]:
# # Show an element of the train_ds with augmentation transformations applied
# samples = [train_ds[0] for _ in range(3)]
# show_samples(samples, ncols=3)
# # No data augmentation applied, at least for now.

In [ ]:
# backbone.__dict__

In [7]:
# Just change the value of selection to try another model

selection = 15
extra_args = {}

if selection == 0:
  model_type = models.mmdet.vfnet
  backbone = model_type.backbones.resnet50_fpn_mstrain_2x

if selection == 1:
  model_type = models.mmdet.retinanet
  backbone = model_type.backbones.resnet50_fpn_1x
  # extra_args['cfg_options'] = { 
  #   'model.bbox_head.loss_bbox.loss_weight': 2,
  #   'model.bbox_head.loss_cls.loss_weight': 0.8,
  #    }

if selection == 2:
  model_type = models.mmdet.faster_rcnn
  backbone = model_type.backbones.resnet101_fpn_2x
  # extra_args['cfg_options'] = { 
  #   'model.roi_head.bbox_head.loss_bbox.loss_weight': 2,
  #   'model.roi_head.bbox_head.loss_cls.loss_weight': 0.8,
  #    }

if selection == 3:
  model_type = models.mmdet.ssd
  backbone = model_type.backbones.ssd300

if selection == 4:
  model_type = models.mmdet.yolox
  backbone = model_type.backbones.yolox_s_8x8

if selection == 5:
  model_type = models.mmdet.yolof
  backbone = model_type.backbones.yolof_r50_c5_8x8_1x_coco

if selection == 6:
  model_type = models.mmdet.detr
  backbone = model_type.backbones.r50_8x2_150e_coco

if selection == 7:
  model_type = models.mmdet.deformable_detr
  backbone = model_type.backbones.twostage_refine_r50_16x2_50e_coco

if selection == 8:
  model_type = models.mmdet.fsaf
  backbone = model_type.backbones.x101_64x4d_fpn_1x_coco

if selection == 9:
  model_type = models.mmdet.sabl
  backbone = model_type.backbones.r101_fpn_gn_2x_ms_640_800_coco

if selection == 10:
  model_type = models.mmdet.centripetalnet
  backbone = model_type.backbones.hourglass104_mstest_16x6_210e_coco

if selection == 11:
  # The Retinanet model is also implemented in the torchvision library
  model_type = models.torchvision.retinanet
  backbone = model_type.backbones.resnet50_fpn

if selection == 12:
  model_type = models.ross.efficientdet
  backbone = model_type.backbones.tf_lite0
  # The efficientdet model requires an img_size parameter
  extra_args['img_size'] = image_size

if selection == 13:
  model_type = models.ultralytics.yolov5
  backbone = model_type.backbones.small
  # The yolov5 model requires an img_size parameter
  extra_args['img_size'] = image_size

if selection == 14:
  model_type = models.ultralytics.yolov5
  backbone = model_type.backbones.small
  # The yolov5 model requires an img_size parameter
  extra_args['img_size'] = image_size

if selection == 15:
    model_type = models.mmdet.mask_rcnn
    backbone = model_type.backbones.resnet50_fpn_1x

if selection == 16:
    model_type = models.mmdet.mask_rcnn
    backbone = model_type.backbones.mask_rcnn_swin_t_p4_w7_fpn_1x_coco

if selection == 17:
    model_type = models.mmdet.yolact
    backbone = model_type.backbones.r101_1x8_coco
    
model_type, backbone, extra_args

(<module 'icevision.models.mmdet.models.mask_rcnn' from '/usr/local/lib/python3.7/dist-packages/icevision/models/mmdet/models/mask_rcnn/__init__.py'>,
 {})

In [ ]:
# Instantiate the model
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(train_parser.class_map), **extra_args) 

# CONFIG:
Questi parametri si possono cambiare mettendoli come extra_args del modello, come fatto nella celle di selezione modello.

In [ ]:
model.cfg.__dict__

In [ ]:
# Data Loaders
train_dl = model_type.train_dl(train_ds, batch_size=2, num_workers=2, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=2, num_workers=2, shuffle=False)

In [ ]:
# show batch
model_type.show_batch(first(valid_dl), ncols=2, denormalize_fn=My_denormalize)

In [ ]:
# Se il modello è di object detection lascerò OD=True, viceverse se fa segmentazione userò OD=False
OD = True
if OD:
    metrics = [COCOMetric(metric_type=COCOMetricType.bbox, print_summary=True)]
else:
    metrics = [COCOMetric(metric_type=COCOMetricType.mask, print_summary=True)

# Training using fastai

In [ ]:
# 4Claudio: cambia la variabile name con i nomi dei modelli che stai usando
wandb.init(project="ScieOttiche", name="centripetal_net_second_run_data_augmented_ULTIMO_DATASET_B3_BAND_SCALE3", reinit=True)

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics, cbs=[WandbCallback(), SaveModelCallback()])

In [ ]:
learn.lr_find()

# For Sparse-RCNN, use lower `end_lr`
# learn.lr_find(end_lr=0.005)

In [ ]:
learn.fine_tune(20, 0.0010, freeze_epochs=5)

In [ ]:
# class LightModel(model_type.lightning.ModelAdapter):
#     def configure_optimizers(self):
#         return SGD(self.parameters(), lr=1e-4)

# light_model = LightModel(model, metrics=metrics)


# trainer = pl.Trainer(max_epochs=20, gpus=1)
# trainer.fit(light_model, train_dl, valid_dl)

In [ ]:
infer_dl = model_type.infer_dl(valid_ds, batch_size=8, shuffle=False)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)

In [ ]:
model_type.show_results(model, valid_ds, detection_threshold=.5, num_samples=8)

In [ ]:
samples_plus_losses, preds, losses_stats = model_type.interp.plot_top_losses(model=model, dataset=valid_ds, sort_by="loss_total", n_samples=4)

In [ ]:
n = 15
show_preds(preds=preds[:n], figsize=(18,10*n))

In [ ]:
from icevision.models.inference_sahi import IceSahiModel

sahimodel = IceSahiModel(model_type=model_type, model=model, class_map=trainParser.class_map, tfms=valid_tfms, confidence_threshold=0.1)

pred = sahimodel.get_sliced_prediction(
                "/content/drive/MyDrive/DATASET/data/Immagini_test/112.png",
                keep_sahi_format=False,
                return_img=True,
                slice_height = 700,
                slice_width = 700,
                overlap_height_ratio = 0.2,
                overlap_width_ratio = 0.2,
            )


In [ ]:
pred["img"]

In [ ]:
img = PIL.Image.open("/content/drive/MyDrive/DATASET/data/Immagini_test/112.png")
pred_dict  = model_type.end2end_detect(img, valid_tfms, model, class_map=class_map, detection_threshold=0.5)